# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jan 3 2023 8:39AM,254109,10,8967688,"Citieffe, Inc.",Released
1,Jan 3 2023 8:36AM,254108,10,0086272005,ISDIN Corporation,Released
2,Jan 3 2023 8:36AM,254108,10,0086272185,ISDIN Corporation,Released
3,Jan 3 2023 8:36AM,254108,10,0086272190,ISDIN Corporation,Released
4,Jan 3 2023 8:36AM,254108,10,0086272191,ISDIN Corporation,Released
5,Jan 3 2023 8:36AM,254108,10,0086272223,ISDIN Corporation,Released
6,Jan 3 2023 8:36AM,254108,10,0086272225,ISDIN Corporation,Released
7,Jan 3 2023 8:36AM,254108,10,0086272231,ISDIN Corporation,Released
8,Jan 3 2023 8:36AM,254108,10,0086272249,ISDIN Corporation,Released
9,Jan 3 2023 8:36AM,254108,10,0086272250,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
11,254103,Released,1
12,254104,Released,1
13,254107,Released,1
14,254108,Released,24
15,254109,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
254103,NaN,1.0
254104,NaN,1.0
254107,NaN,1.0
254108,NaN,24.0
254109,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
254084,0.0,1.0
254087,0.0,1.0
254090,2.0,60.0
254096,0.0,1.0
254097,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
254084,0,1
254087,0,1
254090,2,60
254096,0,1
254097,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,254084,0,1
1,254087,0,1
2,254090,2,60
3,254096,0,1
4,254097,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,254084,,1
1,254087,,1
2,254090,2,60
3,254096,,1
4,254097,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released      int32
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jan 3 2023 8:39AM,254109,10,"Citieffe, Inc."
1,Jan 3 2023 8:36AM,254108,10,ISDIN Corporation
25,Jan 3 2023 8:27AM,254107,19,Eli Lilly and Company
26,Jan 3 2023 8:19AM,254104,20,"Exact-Rx, Inc."
27,Jan 3 2023 8:18AM,254103,20,"Exact-Rx, Inc."
28,Jan 3 2023 8:16AM,254102,20,"Exact-Rx, Inc."
29,Jan 3 2023 8:15AM,254101,20,"Exact-Rx, Inc."
30,Jan 3 2023 8:14AM,254100,20,"Exact-Rx, Inc."
31,Jan 3 2023 8:12AM,254099,20,"Exact-Rx, Inc."
32,Jan 3 2023 8:11AM,254098,20,"Exact-Rx, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Jan 3 2023 8:39AM,254109,10,"Citieffe, Inc.",,1
1,Jan 3 2023 8:36AM,254108,10,ISDIN Corporation,,24
2,Jan 3 2023 8:27AM,254107,19,Eli Lilly and Company,,1
3,Jan 3 2023 8:19AM,254104,20,"Exact-Rx, Inc.",,1
4,Jan 3 2023 8:18AM,254103,20,"Exact-Rx, Inc.",,1
5,Jan 3 2023 8:16AM,254102,20,"Exact-Rx, Inc.",,1
6,Jan 3 2023 8:15AM,254101,20,"Exact-Rx, Inc.",,1
7,Jan 3 2023 8:14AM,254100,20,"Exact-Rx, Inc.",,1
8,Jan 3 2023 8:12AM,254099,20,"Exact-Rx, Inc.",,1
9,Jan 3 2023 8:11AM,254098,20,"Exact-Rx, Inc.",,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Jan 3 2023 8:39AM,254109,10,"Citieffe, Inc.",1,
1,Jan 3 2023 8:36AM,254108,10,ISDIN Corporation,24,
2,Jan 3 2023 8:27AM,254107,19,Eli Lilly and Company,1,
3,Jan 3 2023 8:19AM,254104,20,"Exact-Rx, Inc.",1,
4,Jan 3 2023 8:18AM,254103,20,"Exact-Rx, Inc.",1,
5,Jan 3 2023 8:16AM,254102,20,"Exact-Rx, Inc.",1,
6,Jan 3 2023 8:15AM,254101,20,"Exact-Rx, Inc.",1,
7,Jan 3 2023 8:14AM,254100,20,"Exact-Rx, Inc.",1,
8,Jan 3 2023 8:12AM,254099,20,"Exact-Rx, Inc.",1,
9,Jan 3 2023 8:11AM,254098,20,"Exact-Rx, Inc.",1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jan 3 2023 8:39AM,254109,10,"Citieffe, Inc.",1,
1,Jan 3 2023 8:36AM,254108,10,ISDIN Corporation,24,
2,Jan 3 2023 8:27AM,254107,19,Eli Lilly and Company,1,
3,Jan 3 2023 8:19AM,254104,20,"Exact-Rx, Inc.",1,
4,Jan 3 2023 8:18AM,254103,20,"Exact-Rx, Inc.",1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Jan 3 2023 8:39AM,254109,10,"Citieffe, Inc.",1,NaN
1,Jan 3 2023 8:36AM,254108,10,ISDIN Corporation,24,NaN
2,Jan 3 2023 8:27AM,254107,19,Eli Lilly and Company,1,NaN
3,Jan 3 2023 8:19AM,254104,20,"Exact-Rx, Inc.",1,NaN
4,Jan 3 2023 8:18AM,254103,20,"Exact-Rx, Inc.",1,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jan 3 2023 8:39AM,254109,10,"Citieffe, Inc.",1,0.0
1,Jan 3 2023 8:36AM,254108,10,ISDIN Corporation,24,0.0
2,Jan 3 2023 8:27AM,254107,19,Eli Lilly and Company,1,0.0
3,Jan 3 2023 8:19AM,254104,20,"Exact-Rx, Inc.",1,0.0
4,Jan 3 2023 8:18AM,254103,20,"Exact-Rx, Inc.",1,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,508217,25,0.0
19,508197,61,2.0
20,2540987,10,0.0
22,254084,1,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,508217,25,0.0
1,19,508197,61,2.0
2,20,2540987,10,0.0
3,22,254084,1,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,25,0.0
1,19,61,2.0
2,20,10,0.0
3,22,1,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,25.0
1,19,Released,61.0
2,20,Released,10.0
3,22,Released,1.0
4,10,Executing,0.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,19,20,22
Status,,,,
Executing,0.0,2.0,0.0,0.0
Released,25.0,61.0,10.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,19,20,22
0,Executing,0.0,2.0,0.0,0.0
1,Released,25.0,61.0,10.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,19,20,22
0,Executing,0.0,2.0,0.0,0.0
1,Released,25.0,61.0,10.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()